### Prerequisites

In [1]:
import urllib.request
from selenium import webdriver
import time as time
from tqdm.auto import tqdm
import os
import shutil

### Input username and number of scrolls for scrapping 

In [2]:
username = str(input("Enter IG username : "))
scrolls = int(input("Enter no of scrolls (minimum = 1) [(total images = (no. of scrolls + 1)*16 approximately ] : "))

Enter IG username : skittles
Enter no of scrolls (minimum = 1) [(total images = (no. of scrolls + 1)*16 approximately ] : 3


### Defining variables to store links

In [3]:
download_urls_img = []
download_urls_vid_page = []
download_urls_vid = []
download_urls_carousel_page = []
filename = ""

### Use the webdriver compatible with your browser, mine is for chrome version 83.0.
#### Webdrivers compatible with your browser can be found at <a>https://chromedriver.chromium.org/downloads</a>

In [4]:
browser = webdriver.Chrome(executable_path="webdriver/chromedriver.exe")
browser.get(("https://www.instagram.com/"+username+"/?hl=en"))
time.sleep(2)

### Getting images, videos and carousel links for given number of scrolls

In [5]:
print("Getting links....")
for scroll_lim in tqdm(range(scrolls)):
    time.sleep(3)
    try:    
        links = browser.find_elements_by_xpath("//div[@class = 'v1Nh3 kIKUG  _bz0w']")
        for link in range(len(links)):
            try:
                links[link].find_element_by_xpath(".//div[@class = 'u7YqG']").find_element_by_xpath(".//span[@aria-label = 'Video']")
                if(links[link].find_element_by_xpath(".//a").get_attribute("href") not in download_urls_vid_page):
                    download_urls_vid_page.append(links[link].find_element_by_xpath(".//a").get_attribute("href"))
            except:
                try:
                    links[link].find_element_by_xpath(".//div[@class = 'u7YqG']").find_element_by_xpath(".//span[@aria-label = 'Carousel']")
                    if(links[link].find_element_by_xpath(".//a").get_attribute("href") not in download_urls_carousel_page):
                        download_urls_carousel_page.append(links[link].find_element_by_xpath(".//a").get_attribute("href"))
                except:
                    if(links[link].find_element_by_xpath(".//img[@class = 'FFVAD']").get_attribute("src") not in download_urls_img):
                        download_urls_img.append(links[link].find_element_by_xpath(".//img[@class = 'FFVAD']").get_attribute("src"))
        if(scrolls>1):
            browser.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            try:
                browser.find_element_by_xpath("//div[@class = '_7UhW9   xLCgt       qyrsm         h_zdq uL8Hv     l4b0S    ']").click()
            except:
                pass
    except:
        pass
    
time.sleep(2)

Getting links....


### Decoding video links

In [6]:
print("Decoding video links....")
if(len(download_urls_vid_page) == 0):
    print("No videos found")
else:
    for link in tqdm(range(len(download_urls_vid_page))):
        browser.get(download_urls_vid_page[link])
        time.sleep(2)
        try:
            if(browser.find_element_by_xpath("//video[@class = 'tWeCl']").get_attribute("src") not in download_urls_vid): 
                download_urls_vid.append(browser.find_element_by_xpath("//video[@class = 'tWeCl']").get_attribute("src"))
        except:
            pass
        
time.sleep(2)

Decoding video links....


### Decoding carousel links

In [7]:
print("Decoding carousel links....")
if(len(download_urls_carousel_page) == 0):
    print("No carousels found")
else:
    for link in tqdm(range(len(download_urls_carousel_page))):
            browser.get(download_urls_carousel_page[link])
            time.sleep(2)
            try:
                temp = browser.find_elements_by_xpath("//div[@class = 'Yi5aA ']")
                time.sleep(2)
                for counter in range(len(temp)+1):
                            try:
                                carousel_vid = browser.find_elements_by_xpath("//video[@class = 'tWeCl']")
                                if(carousel_vid):
                                    for vid_carousel_counter in range(len(carousel_vid)):     
                                        if(carousel_vid[vid_carousel_counter].get_attribute("src") not in download_urls_vid): 
                                            download_urls_vid.append(carousel_vid[vid_carousel_counter].get_attribute("src"))

                                else:
                                    carousel_img = browser.find_elements_by_xpath("//img[@class = 'FFVAD']")
                                    for img_carousel_counter in range(len(carousel_img)):
                                        if(carousel_img[img_carousel_counter].get_attribute("src") not in download_urls_img):
                                            download_urls_img.append(carousel_img[img_carousel_counter].get_attribute("src"))

                            except:
                                pass

                            try:
                                browser.find_element_by_xpath("//button[@class = '  _6CZji ']").click()
                                time.sleep(2)
                            except:
                                pass
            except:
                pass

browser.close()

Decoding carousel links....


### Displaying download summary

In [8]:
print("Total photos to download : ", len(download_urls_img))
print("Total videos to download : ", len(download_urls_vid))
print("Total posts : ", len(download_urls_img) + len(download_urls_vid))

Total photos to download :  46
Total videos to download :  10
Total posts :  56


### Creating a folder to store downloaded data under the 'Outputs' directory 

In [9]:
try:
    shutil.rmtree(os.path.join(os.getcwd(), "Outputs", username))
except:
    pass
os.mkdir(os.path.join(os.getcwd(), "Outputs", username))
os.mkdir(os.path.join(os.getcwd(), "Outputs", username, "Images"))
os.mkdir(os.path.join(os.getcwd(), "Outputs", username, "Videos"))

print("Folder created")

Folder created


### Downloading data

In [10]:
counter_img = 0
counter_vid = 0
print("Downloading images....")
if(len(download_urls_img) == 0):
    print("No images found")
else:
    for link in tqdm(download_urls_img):
        print(" ", end='\r')
        counter_img+=1
        filename = "Outputs/" + username + "/Images/image_" + str(counter_img) + ".jpeg"
        try:
            urllib.request.urlretrieve(link, filename)
        except:
            pass
print("Downloading videos....")

if(len(download_urls_vid) == 0):
    print("No videos found")
else:
    for link in tqdm(download_urls_vid):
        print(" ", end='\r')
        counter_vid+=1
        filename = "Outputs/" + username + "/Videos/video_" + str(counter_vid) + ".mp4"
        try:
            urllib.request.urlretrieve(link, filename)
        except:
            pass